In [1]:
from arraytainers import Numpytainer, Jaxtainer
import numpy as np
from numpy import array
import jax
import jax.numpy as jnp
import functools
from jax.numpy import DeviceArray

In [33]:
A[{'a':['c', 'd'], 'b': None}].shape

Numpytainer({'a': {'c': [array([6, 3, 5, 2])], 'd': array([6, 3, 5, 2])}, 'b': array([1, 3, 3, 5, 5])})

In [29]:
a = {'a': {'c':[np.random.rand(6,3,5,2)], 'd':np.random.rand(6,3,5,2)}, 'b': np.random.rand(3,3,5,5), 'e':2}
A = Numpytainer(a)
# A.reshape(A.shape[:1], 15, A.shape[-1:]).shape

In [3]:
a = {'a': np.random.rand(4,3,5,5), 'b': np.random.rand(3,3,5,5)}
A = Jaxtainer(a)
A.reshape(A.shape[:-2], 25).shape

Jaxtainer({'a': DeviceArray([ 4,  3, 25], dtype=int32), 'b': DeviceArray([ 3,  3, 25], dtype=int32)})

In [4]:
a = {'a': np.random.rand(4,3,5), 'b': np.random.rand(3,4,5,5)}
A = Jaxtainer(a)
A.reshape(12, A.shape[2:]).shape

Jaxtainer({'a': DeviceArray([12,  5], dtype=int32), 'b': DeviceArray([12,  5,  5], dtype=int32)})